# Loading and scoring some FashionMNIST images with an ONNX model

Load up the test data from `data/images.h5` (an HDF archive)

In [ ]:
import h5py

file = h5py.File('data/images.h5', 'r')

file.keys()

Get the records into numpy

In [ ]:
import numpy as np

arr = np.array(file['batch'])
arr.shape, arr.dtype

Create an `onnxruntime` inference session object from the model at `data/fmnist.onnx`

In [ ]:
import onnxruntime as rt

session = rt.InferenceSession("data/fmnist.onnx")

for i in session.get_inputs():
    print(i)

Prepare the data for scoring:
* original data is
    * pixel intensities (0-255 as uint8)
    * and shape is (10000, 28, 28) as (batch, h, w)
* we need to convert them to
    * scaled from 0.0 to 1.0 
    * NumPy float32 type
* for scoring, the model expects batches of shape
    * (batch_size, 1, 28, 28) which is (batch_size, channels, h, w)
    * batch_size can be variable

In [ ]:
arr = arr / 255.0
arr.dtype

In [ ]:
arr = arr.reshape(-1, 1, 28,28).astype('float32')

In [ ]:
outputs = session.run(None, {'input': arr })

Note the output is always a `list` ... in our case, we just need one item

In [ ]:
outputs[0].shape

For each image input, we get 10 logit scores. We can argmax these to find MLE clothing item class index.

In [ ]:
outputs[0][0]

In [ ]:
predictions = outputs[0].argmax(axis=1)

predictions[:5]